In [20]:
import tkinter as tk
import os
import sys
from tkinter import ttk
import sqlite3
from openpyxl import Workbook
import subprocess

conn = sqlite3.connect('prod_plan.db')
cursor = conn.cursor()

cursor.execute('''CREATE TABLE IF NOT EXISTS machines
               (id INTEGER PRIMARY KEY AUTOINCREMENT,
               name TEXT)''')

def add_machine(cursor, name):
    cursor.execute("INSERT INTO machines (name) VALUES (?)", (name,))

machines = ["Станок1", "Станок2", "Станок3", "Станок4", "Станок5",
            "Станок6", "Станок7", "Станок8", "Станок9", "Станок10"]

# for machine in machines:
#     add_machine(cursor, machine)

def get_machines_from_db(cursor):
    cursor.execute("SELECT name FROM machines")
    machines = [row[0] for row in cursor.fetchall()]
    return machines

def get_tables_from_db(cursor):
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tables = [row[0] for row in cursor.fetchall()]
    return tables

def get_info_from_db(cursor, table):
    cursor.execute(f"SELECT main FROM {table}")
    info = [row[0] for row in cursor.fetchall()]
    return info

def save_to_excel(machines, file_path):
    workbook = Workbook()
    sheet = workbook.active
    sheet.title = "Machines"
    sheet.append(["Machine Names"])
    for machine in machines:
        sheet.append([machine])
    workbook.save(file_path)

def show_selection_window():
    def on_button_click():
        selected_machine = machine_combobox.get()
        label.config(text="Выбран станок: " + selected_machine)
        submit_button.pack(pady=10)
        submit_button.config(command=lambda: table_machine(selected_machine))

    def table_machine(selected_machine):
        cursor.execute(f'''CREATE TABLE IF NOT EXISTS {selected_machine} 
                   (id INTEGER PRIMARY KEY AUTOINCREMENT,
                   main TEXT, params TEXT, date DATE, hours_kol INT)''')
        # task_day = "02.01.2023"
        cursor.execute("INSERT INTO Станок4 (main, params, date) VALUES (?, ?, ?)", (main_text.get(), param_text.get(), date_text.get(),))

        conn.commit()

    def on_yes_clicked():
        label.config(text="Введена строка: " + main_text.get())
        params_out.config(text="Введен параметр: " + param_text.get())
        date_out.config(text="Введен день: " + date_text.get())
    def on_excel_clicked():
        save_file_path = os.path.abspath("machines.xlsx")
        conn = sqlite3.connect('prod_plan.db')
        cursor = conn.cursor()

        machines = get_machines_from_db(cursor)
        save_to_excel(machines, save_file_path)
        conn.close()

        os.system(f'start excel "{save_file_path}"')

    window = tk.Tk()
    window.title("Выбор станка")
    window_width = 800
    window_height = 700
    screen_width = window.winfo_screenwidth()
    screen_height = window.winfo_screenheight()
    x = (screen_width - window_width) // 2
    y = (screen_height - window_height) // 2 - 50
    geometry_string = f"{window_width}x{window_height}+{x}+{y}"
    window.geometry(geometry_string)
    window.minsize(window_width, window_height)

    machines = get_machines_from_db(cursor)
    tables = get_tables_from_db(cursor)

    machine_combobox = ttk.Combobox(window, values=machines)
    machine_combobox.pack(padx=20, pady=10)


    label = tk.Label(window, text="")
    label.pack(pady=10)

    params_out = tk.Label(window, text="")
    params_out.pack(pady=10)

    date_out = tk.Label(window, text="")
    date_out.pack(pady=10)

    button = tk.Button(window, text="Выбрать", command=on_button_click)
    button.pack(pady=10)

    result_label = tk.Label(window, text="задание:")
    result_label.pack(pady=10)
    
    main_text = tk.StringVar()
    text_entry = tk.Entry(window, textvariable=main_text)
    text_entry.pack(pady=10)

    result_label = tk.Label(window, text="параметры:")
    result_label.pack(pady=10)
    
    param_text = tk.StringVar()
    param_entry = tk.Entry(window, textvariable=param_text)
    param_entry.pack(pady=10)

    result_label = tk.Label(window, text="день:")
    result_label.pack(pady=10)
    
    date_text = tk.StringVar()
    date_entry = tk.Entry(window, textvariable=date_text)
    date_entry.pack(pady=10)

    submit_button = tk.Button(window, text="Подтвердить", command=on_yes_clicked)
    submit_button.pack(pady=10)

    excel_button = tk.Button(window, text="Создать Excel file", command=on_excel_clicked)
    excel_button.pack(pady=10)

    info = get_info_from_db(cursor, "Станок4")
    # print(info)

    tables_label = tk.Label(window, text="main в таблице 'Станок4':")
    tables_label.pack(pady=10)
    tables_text = tk.Text(window, height=5, width=50)
    tables_text.insert(tk.END, "\n".join(tables))
    tables_text.pack(pady=10)


    window.mainloop()

show_selection_window()
conn.close()